**Connect google drive**

In [3]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Fri Mar 18 07:25:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
 619BFC76-F86C-4865-8A1D-3248D4361A65.png
 Artificial_Neural_Network.gdoc
 Classroom
'Colab Notebooks'
 MLM-KHNN_assign1_paper.gdoc
'My Drive'
 Object-Detection---Yolov3-master
 support_vector_machines_succinctly.gdoc
 SVM_detailed.gdoc
 SVM.example2.gdoc
'US  E-commerce records 2020 (1).csv'
'US  E-commerce records 2020 (2).csv'
'US  E-commerce records 2020 (3).csv'
'US  E-commerce records 2020a.xlsx'
'US  E-commerce records 2020.csv'
 yolov3


**1) Clone the Darknet**



In [5]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15412, done.
remote: Total 15412 (delta 0), reused 0 (delta 0), pack-reused 15412
Receiving objects: 100% (15412/15412), 14.05 MiB | 17.90 MiB/s, done.
Resolving deltas: 100% (10354/10354), done.


**2) Compile Darknet using Nvidia GPU**


In [6]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [7]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [8]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [9]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [10]:
!echo "Koala" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [11]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-03-18 07:27:19--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  56.6MB/s    in 2.7s    

2022-03-18 07:27:22 (56.6 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [12]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/bob-walker-wqyjvlpXwxc-unsplash.txt  
  inflating: data/obj/christopher-alvarenga-3G1C_MvxlKs-unsplash.jpg  
  inflating: data/obj/christopher-alvarenga-3G1C_MvxlKs-unsplash.txt  
  inflating: data/obj/classes.txt    
  inflating: data/obj/craig-garrett-kHTMQ8ngLVs-unsplash.jpg  
  inflating: data/obj/craig-garrett-kHTMQ8ngLVs-unsplash.txt  
  inflating: data/obj/craig-garrett-vhTkfZvKbPM-unsplash.jpg  
  inflating: data/obj/craig-garrett-vhTkfZvKbPM-unsplash.txt  
  inflating: data/obj/craig-manners-0RWZ65T3Kxo-unsplash.jpg  
  inflating: data/obj/craig-manners-0RWZ65T3Kxo-unsplash.txt  
  inflating: data/obj/cris-saur-GNUcUx-iObg-unsplash (1).jpg  
  inflating: data/obj/cris-saur-GNUcUx-iObg-unsplash (1).txt  
  inflating: data/obj/cris-saur-GNUcUx-iObg-unsplash.jpg  
  inflating: data/obj/cris-saur-GNUcUx-iObg-unsplash.txt  
  inflating: data/obj/dan-kb-_4duU16GtIw-unsplash.jpg  
  inflating: data/obj/dan-kb-_4duU16GtIw-unsp

In [13]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.439236 0.571759 0.342448 0.688079

['0', '0.439236', '0.571759', '0.342448', '0.688079']
0 data/obj/pexels-pixabay-532543.txt
0 0.439236 0.571759 0.342448 0.688079
0 0.416500 0.396652 0.515500 0.441603

['0', '0.416500', '0.396652', '0.515500', '0.441603']
1 data/obj/hugo-kruip-N0HdQGHuY4I-unsplash.txt
0 0.416500 0.396652 0.515500 0.441603
0 0.507062 0.533608 0.394774 0.603538

['0', '0.507062', '0.533608', '0.394774', '0.603538']
2 data/obj/dillon-pena-At7EPUv3gKc-unsplash.txt
0 0.507062 0.533608 0.394774 0.603538
0 0.569500 0.334750 0.232667 0.352000

['0', '0.569500', '0.334750', '0.232667', '0.352000']
3 data/obj/nadir-syzygy-KNpQAX6Q6cU-unsplash.txt
0 0.569500 0.334750 0.232667 0.352000
0 0.340685 0.498244 0.162168 0.644557

['0', '0.340685', '0.498244', '0.162168', '0.644557']
4 data/obj/zizhang-cheng-kEcf0AXtUAU-unsplash.txt
0 0.340685 0.498244 0.162168 0.644557
0 0.520657 0.531014 0.373941 0.576651

['0', '0.520657', '0.531014', '0.373941', '0.576651']
5 data/obj/dillon-pen

In [14]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/annie-spratt-nKH8-IVPy9M-unsplash.jpg', 'data/obj/samira-mohajerani-Y7Q2shLO0ag-unsplash (1).jpg', 'data/obj/nadir-syzygy-KNpQAX6Q6cU-unsplash.jpg', 'data/obj/parth-patel-ESiIfHBCxDc-unsplash.jpg', 'data/obj/pexels-doug-brown-1770705.jpg', 'data/obj/hugo-kruip-N0HdQGHuY4I-unsplash (1).jpg', 'data/obj/wim-bollen-qvWx2bzsIv8-unsplash.jpg', 'data/obj/angus-gray-al2BO4QBRRM-unsplash.jpg', 'data/obj/miranda-garside-dNYqZMxWbLQ-unsplash.jpg', 'data/obj/szabolcs-toth-mgr9YfGgXLU-unsplash (1).jpg', 'data/obj/greg-spearritt-SqonhHMYLqQ-unsplash.jpg', 'data/obj/ellena-mcguinness-xsUBd6wwuOI-unsplash (2).jpg', 'data/obj/alephx-phxale-jxCywZPPiFA-unsplash.jpg', 'data/obj/parth-patel-ESiIfHBCxDc-unsplash (1).jpg', 'data/obj/joseph-perez-EH3s7UzsmjI-unsplash.jpg', 'data/obj/simon-maisch-EHFTa1BgMHk-unsplash.jpg', 'data/obj/szabolcs-toth-mgr9YfGgXLU-unsplash (2).jpg', 'data/obj/kenzie-broad-jWczCsgTWGc-unsplash.jpg', 'data/obj/angela-matijczak-3kfSI8z8ytE-unsplash (1).jpg', 'data/obj/bob-w

In [15]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
 total_bbox = 41381, rewritten_bbox = 0.923129 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.665255), count: 4, class_loss = 0.476984, iou_loss = 0.403302, total_loss = 0.880286 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.001191, iou_loss = 0.000000, total_loss = 0.001191 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000016, iou_loss = 0.000000, total_loss = 0.000016 
 total_bbox = 41385, rewritten_bbox = 0.923040 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.733748), count: 3, class_loss = 0.322477, iou_loss = 0.219251, total_loss = 0.541728 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.465040), count: 1, class_loss = 0.243368, iou_loss = 0.486283, total_loss = 0.729650 
v3 (mse 